In [5]:
from PIL import Image
import pytesseract
from pytesseract import Output
import cv2
import numpy as np


class Paragraph:

    def __init__(self, img, para_info):
        self._para_info = para_info
        self._img = img
        self._lines_coords = None
        self._lines_ct = None
        self._paragr_coord = None
        
    @property
    def lines_ct(self):
        if self._lines_ct is None:
            self._lines_ct = self._para_info[0]
        return self._lines_ct

    @property
    def lines_coords(self):
        if self._lines_coords is None:
            self._lines_coords = self._para_info[1]
        return self._lines_coords  

    @property
    def paragraph_content(self):
        paragraph = ''
        for line in self.lines_ct:
            for word in line:
                paragraph += word + ' '
        return paragraph
    
    @property
    def paragraph_coord(self):
        if self._paragr_coord is None:
            x1 = min([f[0] for f in self.lines_coords])
            y1 = self.lines_coords[0][1]
            x2 = max([f[2] for f in self.lines_coords])
            y2 = self.lines_coords[-1][3]
            self._paragr_coord = [x1, y1, x2, y2]
        return self._paragr_coord
    
    @property
    def paragraph_img_crop(self):
        x1, y1, x2, y2 = self.paragraph_coord
        paragr_img = self._img[y1:y2, x1:x2]
        return paragr_img


class Line:

    def __init__(self, img, line_coord, line_ct):
        self._img = img
        self._line_coord = line_coord
        self._line_ct = line_ct
    
    @property
    def line_content(self):
        line = ''
        for word in self._line_ct:
            line += word + ' '
        return line
    
    @property
    def line_img_crop(self):
        x1, y1, x2, y2 = self._line_coord
        crop = self._img[y1:y2, x1:x2]
        return crop
    

img = cv2.imread('../test_img/15d605a9-7ff4-4c1c-b41a-0a5e8f9262ba.jpg')
text_info = pytesseract.image_to_data(
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 
    lang='vie', 
    output_type=Output.DICT, 
    )
remove_ids = []
for id, text in enumerate(text_info['text']):
    if text  == "":
        remove_ids.append(id)
for key in text_info.keys():
    for id, val in enumerate(remove_ids):
        if id == 0:
            del text_info[key][val]
        else:
            del text_info[key][val - id]
extract_ids = {}
for ch_id in range(text_info['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_info['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_info['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_info['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
# print(extract_ids)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_info['left'][begin_id], text_info['top'][begin_id], text_info['width'][begin_id], text_info['height'][begin_id]
            e_x, e_y, e_w, e_h = text_info['left'][end_id], text_info['top'][end_id], text_info['width'][end_id], text_info['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_info['text'][word_idx])
            pr_lines_ct.append(line_ct)
        # print(pr_lines_ct)
        # print('.......................')
        need_to_remove = len(pr_lines_ct[0]) == 1 and len(pr_lines_ct[0][0]) <= 3 and pr_lines_ct[0][0].isdecimal()
        if need_to_remove: 
            pass
        else: 
            paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
print(paragraphs_infor)
# exam_para = Paragraph(img, paragraphs_infor[1])
# exam_line_coord = exam_para.lines_coords[0]
# exam_line_ct = exam_para.lines_ct[0]
# exam_line = Line(img, exam_line_coord, exam_line_ct)
# print(exam_line._line_coord)
# cv2.imwrite('para.jpg', exam_para.paragraph_img_crop)



        

[[[['Mâu', 'thuẫn', 'giữa', 'các', 'tầng', 'lớp', 'nhân', 'dân', 'Ấn', 'Độ', 'với', 'thực', 'dân', 'Anh', 'ngày', 'càng', 'sâu'], ['sắc,', 'làm', 'bùng', 'nổ', 'nhiều', 'cuộc', 'khởi', 'nghĩa', 'chống', 'Anh.', 'Tiêu', 'biểu', 'cho', 'phong', 'trào', 'đấu'], ['tranh', 'giải', 'phóng', 'dân', 'tộc', 'của', 'Ấn', 'Độ', 'vào', 'nửa', 'sau', 'thế', 'kỉ', 'XIX', 'là', 'cuộc', 'khởi', 'nghĩa', 'của'], ['quân', 'Xipay', 'và', 'nhân', 'dân', 'ở', 'Mi-rút.']], [[316, 29, 1328, 51], [262, 73, 1328, 102], [262, 124, 1330, 148], [261, 174, 700, 203]]], [[['Xipay', 'là', 'tên', 'gọi', 'những', 'đơn', 'vị', 'bình', 'lính', 'người', 'Ấn', 'Độ', 'trong', 'quân', 'đội', 'của'], ['thực', 'dân', 'Anh.', 'Mặc', 'dù', 'là', 'một', 'trong', 'những', 'công', 'cụ', 'xâm', 'lược', 'và', 'thống', 'trị'], ['của', 'thực', 'dân,', 'binh', 'lính', 'người', 'Ấn', 'vẫn', 'bị', 'sĩ', 'quan', 'người', 'Anh', 'đối', 'xử', 'tàn', 'tệ.'], ['Tỉnh', 'thần', 'dân', 'tộc', 'và', 'tín', 'ngưỡng', 'của', 'họ', 'luôn', 'bị', 'xú

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../book_test/VanBanGoc_108-2008-TT-BTC_108-2008-TT-BTC/vbg_7.jpg'
img = cv2.imread(path)
## option 1 ##
# gray_img = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
# binary_img = cv2.threshold(gray_img.copy(), 255, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
##option 2 ##
def compute_hist(img):
    hist = np.zeros((256,), np.uint8)
    h, w = img.shape[:2]
    for i in range(h):
        for j in range(w):
            hist[img[i][j]] += 1
    return hist

def equal_hist(hist):
    cumulator = np.zeros_like(hist, np.float64)
    for i in range(len(cumulator)):
        cumulator[i] = hist[:i].sum()
    new_hist = (cumulator - cumulator.min())/(cumulator.max() - cumulator.min()) * 255
    new_hist = np.uint8(new_hist)
    return new_hist

hist = compute_hist(img).ravel()
new_hist = equal_hist(hist)
h, w = img.shape[:2]
for i in range(h):
    for j in range(w):
        img[i,j] = new_hist[img[i,j]]

kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1 , -1]])
sharpened_img = cv2.filter2D(img.copy(), -1, kernel)
blur_img = cv2.GaussianBlur(sharpened_img, (3, 3), 0)

text = pytesseract.image_to_string(Image.fromarray(blur_img), lang='vie')[:-1]
print(text)



In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../book_test/VanBanGoc_108-2008-TT-BTC_108-2008-TT-BTC/vbg_7.jpg'
img = cv2.imread(path)

def adjust_image_gamma_lookuptable(image, gamma=1.0):
    table = np.array([((i / 255.0) ** gamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

adjusted_high_img = adjust_image_gamma_lookuptable(img, 5)
# kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1 , -1]])
# sharpened_img = cv2.filter2D(adjusted_high_img.copy(), -1, kernel)
# blur_img = cv2.GaussianBlur(sharpened_img, (3, 3), 0)
text = pytesseract.image_to_string(Image.fromarray(adjusted_high_img), lang='vie')[:-1]
print(text)


In [34]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../book_test/VanBanGoc_108-2008-TT-BTC_108-2008-TT-BTC/vbg_7.jpg'
img = cv2.imread(path)

def adjust_image_gamma_lookuptable(image, gamma=1.0):
    table = np.array([((i / 255.0) ** gamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

adjusted_high_img = adjust_image_gamma_lookuptable(img, 5)
# kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1 , -1]])
# sharpened_img = cv2.filter2D(adjusted_high_img.copy(), -1, kernel)
# blur_img = cv2.GaussianBlur(sharpened_img, (3, 3), 0)
text = pytesseract.image_to_string(Image.fromarray(adjusted_high_img), lang='vie')[:-1]
print(text)


a) Chỉ chuyển nguồn sang ngân sách năm sau theo quy định gồm:

- Số dư dự toán được chuyển sang ngân sách năm sau;

- Số dư tạm ứng được chuyên sang ngân sách năm sau;

- Số dư tài khoản tiền gửi kinh phí ngân sách (gồm cả tài khoản tiền gửi kinh
phí uỷ quyền - nếu có) được chuyen sang ngân sách năm sau;

- Số dư dự toán của các cấp ngân sách được cấp có thẩm quyền cho phép
hoặc theo chế độ quy định được chuyen sang ngân sách năm sau như: Dự phòng,
dự toán chưa phân bồ (nếu có), nguồn von phát hành trái phiếu Chính phủ chưa sử
dụng và số tăng thu so dự toán của các cấp ngân sách.

b) Việc hạch toán chuyển nguon sang ngân sách năm sau thực hiện như sau:

Căn cứ các nhiệm vụ và tong số kinh phí được chuyên sang ngân sách năm
sau theo quy định tại Điểm 2, Đ1em 3, Đ1em 5 và Điểm 7, Muc I Thong tư này, cơ
quan tài chính (Uỷ ban nhân dân cap xã đối với ngân sách cấp xã) có văn bản gửi
Kho bạc Nhà nước nơi giao dịch đề Kho bạc Nhà nước hạch toán chuyen nguôn
sang ngân sách năm sau theo Mục lụ

Remove non-string


In [4]:
text_info = pytesseract.image_to_data(Image.fromarray(img), lang='vie', output_type=Output.DICT)
remove_ids = []
for id, text in enumerate(text_info['text']):
    if text  == "":
        remove_ids.append(id)
for key in text_info.keys():
    for id, val in enumerate(remove_ids):
        if id == 0:
            del text_info[key][val]
        else:
            del text_info[key][val - id]

# only_text = pytesseract.image_to_string(Image.fromarray(img), lang='vie')
# text_lines = only_text.split("\n")[:-1]
# text_lines_no_space = [f.split(' ') for f in text_lines]
# for text_line in text_lines_no_space:
# block_text = pytesseract.image_to_string(Image.fromarray(img), lang='vie')[:-1].replace('\n', ' ')
############################
setted_num_line = list(set(text_info['line_num']))
line_word_coords = []
word_id_line = {}
# for line_num in setted_num_line:
#     word_id_line[line_num] = []
#     for i, num in enumerate(text_info['line_num']):
#         if num == line_num:
#             word_id_line[line_num].append(i)
# for key in word_id_line.keys():
#     begin_id = word_id_line[key][0]
#     end_id = word_id_line[key][-1]
#     b_x, b_y, b_w, b_h = text_info['left'][begin_id], text_info['top'][begin_id], text_info['width'][begin_id], text_info['height'][begin_id]
#     e_x, e_y, e_w, e_h = text_info['left'][end_id], text_info['top'][end_id], text_info['width'][end_id], text_info['height'][end_id]
#     line_word_coords.append([(b_x, b_y, b_w, b_h), (e_x, e_y, e_w, e_h)])
# line_img_crop = []
# c = 0
# for line_coord in line_word_coords:
#     x1 = line_coord[0][0]
#     y1 = min(line_coord[0][1], line_coord[1][1])
#     x2 = line_coord[1][0] + line_coord[1][2]
#     y2 = max((line_coord[0][1] + line_coord[0][3]), (line_coord[1][1] + line_coord[1][3]))
#     line_crop = img[y1:y2, x1:x2]
#     cv2.imwrite('../test_img/u{}.jpg'.format(c), line_crop)
#     c += 1

[1, 2, 3, 4, 5, 6, 7]


In [31]:
extract_ids = {1: {1: {1: [1]}}}
list(extract_ids.keys())[-1]

1

In [3]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = './crop_imgs/129/1.jpg'
img = cv2.imread(path)

def merge_content(lines_ct: list):
    content = ""
    for line in lines_ct:
        for word in line:
            content += word + ' '
    return content

text_infor = pytesseract.image_to_data(Image.fromarray(img), lang='vie', output_type=Output.DICT)
remove_ids = []
for id, conf in enumerate(text_infor['conf']):
    if conf  == '-1' or conf < 40:
        remove_ids.append(id)
for key in text_infor.keys():
    for id, val in enumerate(remove_ids):
        del text_infor[key][val - id]
        
extract_ids = {}
for ch_id in range(text_infor['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_infor['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_infor['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_infor['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_infor['left'][begin_id], text_infor['top'][begin_id], text_infor['width'][begin_id], text_infor['height'][begin_id]
            e_x, e_y, e_w, e_h = text_infor['left'][end_id], text_infor['top'][end_id], text_infor['width'][end_id], text_infor['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_infor['text'][word_idx])
            pr_lines_ct.append(line_ct)
        paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
    
for para in paragraphs_infor:
    print(merge_content(para[0]))

Do you know that this job involves some travelling? 
Anh có biết là công biệc này đôi khi cần phải đi lại không? Im prepared to travel if necessary. As I stated in my letter, I have my own car. 
Tôi sẵn sàng đi đây đó nếu cần. Như tôi đã nêu trong thu, lôi có xe hơi riêng. 


In [5]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output
import os

path = './crop_imgs/'
count = 0
folder_names = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
img_names = [f for f in os.listdir(path) if f.find('jpg') != -1 or f.find('.png') != -1]
img_paths = [os.path.join(path, f) for f in img_names]
for name in folder_names:
    dir_path = os.path.join(path, name)
    file_names = os.listdir(dir_path)
    for sub_name in file_names:
        sub_path = os.path.join(dir_path, sub_name)
print(folder_names[-1])
# def merge_content(lines_ct: list):
#     content = ""
#     for line in lines_ct:
#         for word in line:
#             content += word + ' '
#     return content

     
# for path in img_paths[:1]:
#     print(path)
#     print('////////////////////////')
#     img = cv2.imread(path)

#     text_infor = pytesseract.image_to_data(Image.fromarray(img), lang='vie', output_type=Output.DICT)
#     remove_ids = []
#     for id, conf in enumerate(text_infor['conf']):
#         if conf  == '-1' or conf < 40:
#             remove_ids.append(id)
#     for key in text_infor.keys():
#         for id, val in enumerate(remove_ids):
#             del text_infor[key][val - id]
#     extract_ids = {}
#     for ch_id in range(text_infor['block_num'].__len__()):
#         if not bool(extract_ids):
#             extract_ids = {1: {1: {1: [ch_id]}}}
#         else:
#             max_block_num = (list(extract_ids.keys()))[-1]
#             max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
#             max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
#             if text_infor['block_num'][ch_id] != max_block_num:
#                 extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
#             else: 
#                 if text_infor['par_num'][ch_id] != max_par_num:
#                     extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
#                 else:
#                     if text_infor['line_num'][ch_id] != max_line_num:
#                         extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
#                     else:
#                         extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
#     paragraphs_infor = []
#     for bl in extract_ids.keys():
#         for par in extract_ids[bl].keys():
#             pr_lines_ct = []
#             pr_lines_coords = []
#             for line in extract_ids[bl][par].keys():
#                 begin_id = extract_ids[bl][par][line][0]
#                 end_id = extract_ids[bl][par][line][-1]
#                 b_x, b_y, b_w, b_h = text_infor['left'][begin_id], text_infor['top'][begin_id], text_infor['width'][begin_id], text_infor['height'][begin_id]
#                 e_x, e_y, e_w, e_h = text_infor['left'][end_id], text_infor['top'][end_id], text_infor['width'][end_id], text_infor['height'][end_id]
#                 x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
#                 pr_lines_coords.append([x1, y1, x2, y2])
#                 line_ct = []
#                 for word_idx in extract_ids[bl][par][line]:
#                     line_ct.append(text_infor['text'][word_idx])
#                 pr_lines_ct.append(line_ct)
#             paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
# for para in paragraphs_infor:
#     print(merge_content(para[0]))
    


what
